In [62]:
import numpy as np
from pymatgen import Composition, Element
#build NN using tensorflow
#use one hiddlen layer, calculate the error and loss
#split the dataset to about 80% 10% 10%(or less, boundry condition) as training, validation and test set.

In [18]:
#get main group elements list
with open("elements_list_main.txt", "r") as text_file:
    a=text_file.read()
main_group_list=a.split()

In [19]:
#load vectors(from SVD_20)
F=np.loadtxt('SVD_main_group.txt')

In [15]:
#load formation energy as a dictionary
with open("formation_energy_dataset.txt", "r") as text_file:
    diction=text_file.read()
di=diction.split()
formation_dict={}
for i in range(len(di)):
    if i%2==0:
        formation_dict[di[i]]=di[i+1]

In [88]:
#build input vector(20vectors*4elements=80vectors)
#vector_input.shape=[5628,20]
vector_input=np.zeros((len(formation_dict),F.shape[1]*4))

formula_number=0
test_string_list=[]
for formula in formation_dict:
    vector=np.zeros((F.shape[1]*4))
    
    compound=Composition(formula)
    element_list=[]
    for element in compound:
        element_list.append(str(element))
    
    #sort element_list by the number of elements
    element_list_sorted=sorted(element_list,key=lambda s: compound[s])
    #build vector
    #test sort result
    formula_string=""
    element_order=0
    for element in element_list_sorted:
        formula_string+=str(element)+str(int(compound[element]))
        for j in range(len(main_group_list)):
            if str(element)==main_group_list[j]:
                #print(j)
                #print(F[j,:])
                vector[element_order*F.shape[1]:(element_order+1)*F.shape[1]]=F[j,:]
                #print(len(vector))
        element_order+=1
    #print(vector)
    vector_input[formula_number,:]=vector
    test_string_list.append(formula_string)
    formula_number+=1

#build success, input vector=vector_input 5628*80

In [86]:
vector_input.shape

(5628, 80)

In [108]:
#build formula and energy list
formula_list=[]
energy_list=[]
for formula in formation_dict:
    energy_list.append(formation_dict[formula])

for formula in test_string_list:
    formula_list.append(formula)
#build success, energy list=energy_list 5628
#build success, formula list=formula_list 5628
print(len(energy_list))
print(len(formula_list))

5628
5628


In [127]:
##store the dataset in file
#1.store the formula_list and energy_list to formation_energy_formula.txt

with open("FE_TRUE.txt", "w") as text_file:
    for i in range(len(formula_list)):
        text_file.write(formula_list[i]+" "+energy_list[i]+"\n")
        
#2. store the vector_input in SVD_20_INPUT.txt
np.savetxt("SVD_20_INPUT.txt",vector_input)

In [110]:
#build index number list
index_list=list(range(len(formula_list)))
#shuffle index_list
np.random.shuffle(index_list)

split_percent=[0.8,0.1,0.1]
boundry1=int(len(index_list)*split_percent[0])
boundry2=int(len(index_list)*(split_percent[1]+split_percent[0]))

#build split-index matrix
traning_index=index_list[:boundry1]
validation_index=index_list[boundry1:boundry2]
test_index=index_list[boundry2:]

#test split lenth
print(len(traning_index))
print(len(validation_index))
print(len(test_index))

4502
563
563


In [52]:
#vector=vector_input 5628*80
#energy list=energy_list 5628 (string array, to be transformed to numpy array, dtype=np.double)
#formula list=formula_list 5628

In [111]:
training_x=vector_input[traning_index,:]
validation_x=vector_input[validation_index,:]
test_x=vector_input[test_index,:]

#trasform energy_list(string list) to numpy, dtype=np.double
energy_array=np.array(energy_list,dtype=np.double)

training_y=energy_array[traning_index]
validation_y=energy_array[validation_index]
test_y=energy_array[test_index]

#training_x
#training_y
#validation_x
#validation_y
#test_x
#test_y

In [130]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam',
              loss='mae',
              metrics=['mae', 'acc'])

history=model.fit(training_x, training_y, validation_data=[validation_x,validation_y],epochs=1000)
evaluation=model.evaluate(validation_x, validation_y)

Train on 4502 samples, validate on 563 samples
Epoch 1/1000
4502/4502 [==============================] - 1s 242us/step - loss: 7.4785 - mean_absolute_error: 7.4785 - acc: 0.0000e+00 - val_loss: 7.1548 - val_mean_absolute_error: 7.1548 - val_acc: 0.0000e+00
Epoch 2/1000
4502/4502 [==============================] - 0s 79us/step - loss: 6.5174 - mean_absolute_error: 6.5174 - acc: 0.0000e+00 - val_loss: 6.4113 - val_mean_absolute_error: 6.4113 - val_acc: 0.0000e+00
Epoch 3/1000
4502/4502 [==============================] - 0s 69us/step - loss: 5.8739 - mean_absolute_error: 5.8739 - acc: 0.0000e+00 - val_loss: 5.8236 - val_mean_absolute_error: 5.8236 - val_acc: 0.0000e+00
Epoch 4/1000
4502/4502 [==============================] - 0s 67us/step - loss: 5.3479 - mean_absolute_error: 5.3479 - acc: 0.0000e+00 - val_loss: 5.3686 - val_mean_absolute_error: 5.3686 - val_acc: 0.0000e+00
Epoch 5/1000
4502/4502 [==============================] - 0s 70us/step - loss: 4.9605 - mean_absolute_error: 4.9605 

Epoch 40/1000
4502/4502 [==============================] - 0s 101us/step - loss: 3.2582 - mean_absolute_error: 3.2582 - acc: 0.0000e+00 - val_loss: 3.5221 - val_mean_absolute_error: 3.5221 - val_acc: 0.0000e+00
Epoch 41/1000
4502/4502 [==============================] - 0s 106us/step - loss: 3.2390 - mean_absolute_error: 3.2390 - acc: 0.0000e+00 - val_loss: 3.5007 - val_mean_absolute_error: 3.5007 - val_acc: 0.0000e+00
Epoch 42/1000
4502/4502 [==============================] - 1s 119us/step - loss: 3.2203 - mean_absolute_error: 3.2203 - acc: 0.0000e+00 - val_loss: 3.4820 - val_mean_absolute_error: 3.4820 - val_acc: 0.0000e+00
Epoch 43/1000
4502/4502 [==============================] - 0s 82us/step - loss: 3.2020 - mean_absolute_error: 3.2020 - acc: 0.0000e+00 - val_loss: 3.4695 - val_mean_absolute_error: 3.4695 - val_acc: 0.0000e+00
Epoch 44/1000
4502/4502 [==============================] - 1s 127us/step - loss: 3.1806 - mean_absolute_error: 3.1806 - acc: 0.0000e+00 - val_loss: 3.4433 - 

Epoch 79/1000
4502/4502 [==============================] - 0s 91us/step - loss: 2.5664 - mean_absolute_error: 2.5664 - acc: 0.0000e+00 - val_loss: 2.8162 - val_mean_absolute_error: 2.8162 - val_acc: 0.0000e+00
Epoch 80/1000
4502/4502 [==============================] - 1s 144us/step - loss: 2.5533 - mean_absolute_error: 2.5533 - acc: 0.0000e+00 - val_loss: 2.8074 - val_mean_absolute_error: 2.8074 - val_acc: 0.0000e+00
Epoch 81/1000
4502/4502 [==============================] - 0s 94us/step - loss: 2.5415 - mean_absolute_error: 2.5415 - acc: 0.0000e+00 - val_loss: 2.8031 - val_mean_absolute_error: 2.8031 - val_acc: 0.0000e+00
Epoch 82/1000
4502/4502 [==============================] - 1s 212us/step - loss: 2.5305 - mean_absolute_error: 2.5305 - acc: 0.0000e+00 - val_loss: 2.7733 - val_mean_absolute_error: 2.7733 - val_acc: 0.0000e+00
Epoch 83/1000
4502/4502 [==============================] - 1s 132us/step - loss: 2.5201 - mean_absolute_error: 2.5201 - acc: 0.0000e+00 - val_loss: 2.7674 - v

Epoch 118/1000
4502/4502 [==============================] - 0s 97us/step - loss: 2.2312 - mean_absolute_error: 2.2312 - acc: 0.0000e+00 - val_loss: 2.4359 - val_mean_absolute_error: 2.4359 - val_acc: 0.0000e+00
Epoch 119/1000
4502/4502 [==============================] - 0s 75us/step - loss: 2.2223 - mean_absolute_error: 2.2223 - acc: 0.0000e+00 - val_loss: 2.4281 - val_mean_absolute_error: 2.4281 - val_acc: 0.0000e+00
Epoch 120/1000
4502/4502 [==============================] - 0s 90us/step - loss: 2.2200 - mean_absolute_error: 2.2200 - acc: 0.0000e+00 - val_loss: 2.4222 - val_mean_absolute_error: 2.4222 - val_acc: 0.0000e+00
Epoch 121/1000
4502/4502 [==============================] - 0s 63us/step - loss: 2.2163 - mean_absolute_error: 2.2163 - acc: 0.0000e+00 - val_loss: 2.4131 - val_mean_absolute_error: 2.4131 - val_acc: 0.0000e+00
Epoch 122/1000
4502/4502 [==============================] - 0s 74us/step - loss: 2.2065 - mean_absolute_error: 2.2065 - acc: 0.0000e+00 - val_loss: 2.4055 -

Epoch 157/1000
4502/4502 [==============================] - 0s 59us/step - loss: 1.9985 - mean_absolute_error: 1.9985 - acc: 0.0000e+00 - val_loss: 2.1914 - val_mean_absolute_error: 2.1914 - val_acc: 0.0000e+00
Epoch 158/1000
4502/4502 [==============================] - 0s 91us/step - loss: 1.9966 - mean_absolute_error: 1.9966 - acc: 0.0000e+00 - val_loss: 2.1797 - val_mean_absolute_error: 2.1797 - val_acc: 0.0000e+00
Epoch 159/1000
4502/4502 [==============================] - 0s 67us/step - loss: 1.9876 - mean_absolute_error: 1.9876 - acc: 0.0000e+00 - val_loss: 2.1770 - val_mean_absolute_error: 2.1770 - val_acc: 0.0000e+00
Epoch 160/1000
4502/4502 [==============================] - 0s 96us/step - loss: 1.9821 - mean_absolute_error: 1.9821 - acc: 0.0000e+00 - val_loss: 2.1625 - val_mean_absolute_error: 2.1625 - val_acc: 0.0000e+00
Epoch 161/1000
4502/4502 [==============================] - 0s 74us/step - loss: 1.9774 - mean_absolute_error: 1.9774 - acc: 0.0000e+00 - val_loss: 2.1637 -

...

Epoch 816/1000
4502/4502 [==============================] - 0s 63us/step - loss: 1.2392 - mean_absolute_error: 1.2392 - acc: 0.0000e+00 - val_loss: 1.5620 - val_mean_absolute_error: 1.5620 - val_acc: 0.0000e+00
Epoch 817/1000
4502/4502 [==============================] - 0s 58us/step - loss: 1.2427 - mean_absolute_error: 1.2427 - acc: 0.0000e+00 - val_loss: 1.5515 - val_mean_absolute_error: 1.5515 - val_acc: 0.0000e+00
Epoch 818/1000
4502/4502 [==============================] - 0s 66us/step - loss: 1.2439 - mean_absolute_error: 1.2439 - acc: 0.0000e+00 - val_loss: 1.5733 - val_mean_absolute_error: 1.5733 - val_acc: 0.0000e+00
Epoch 819/1000
4502/4502 [==============================] - 0s 58us/step - loss: 1.2405 - mean_absolute_error: 1.2405 - acc: 0.0000e+00 - val_loss: 1.5573 - val_mean_absolute_error: 1.5573 - val_acc: 0.0000e+00
Epoch 820/1000
4502/4502 [==============================] - 0s 60us/step - loss: 1.2409 - mean_absolute_error: 1.2409 - acc: 0.0000e+00 - val_loss: 1.5701 -

Epoch 855/1000
4502/4502 [==============================] - 0s 60us/step - loss: 1.2307 - mean_absolute_error: 1.2307 - acc: 0.0000e+00 - val_loss: 1.5618 - val_mean_absolute_error: 1.5618 - val_acc: 0.0000e+00
Epoch 856/1000
4502/4502 [==============================] - 0s 61us/step - loss: 1.2325 - mean_absolute_error: 1.2325 - acc: 0.0000e+00 - val_loss: 1.5482 - val_mean_absolute_error: 1.5482 - val_acc: 0.0000e+00
Epoch 857/1000
4502/4502 [==============================] - 0s 58us/step - loss: 1.2275 - mean_absolute_error: 1.2275 - acc: 0.0000e+00 - val_loss: 1.5365 - val_mean_absolute_error: 1.5365 - val_acc: 0.0000e+00
Epoch 858/1000
4502/4502 [==============================] - 0s 66us/step - loss: 1.2314 - mean_absolute_error: 1.2314 - acc: 0.0000e+00 - val_loss: 1.5379 - val_mean_absolute_error: 1.5379 - val_acc: 0.0000e+00
Epoch 859/1000
4502/4502 [==============================] - 0s 71us/step - loss: 1.2281 - mean_absolute_error: 1.2281 - acc: 0.0000e+00 - val_loss: 1.5390 -

Epoch 894/1000
4502/4502 [==============================] - 0s 70us/step - loss: 1.2194 - mean_absolute_error: 1.2194 - acc: 0.0000e+00 - val_loss: 1.5297 - val_mean_absolute_error: 1.5297 - val_acc: 0.0000e+00
Epoch 895/1000
4502/4502 [==============================] - 0s 62us/step - loss: 1.2142 - mean_absolute_error: 1.2142 - acc: 0.0000e+00 - val_loss: 1.5365 - val_mean_absolute_error: 1.5365 - val_acc: 0.0000e+00
Epoch 896/1000
4502/4502 [==============================] - 0s 59us/step - loss: 1.2210 - mean_absolute_error: 1.2210 - acc: 0.0000e+00 - val_loss: 1.5219 - val_mean_absolute_error: 1.5219 - val_acc: 0.0000e+00
Epoch 897/1000
4502/4502 [==============================] - 0s 65us/step - loss: 1.2165 - mean_absolute_error: 1.2165 - acc: 0.0000e+00 - val_loss: 1.5336 - val_mean_absolute_error: 1.5336 - val_acc: 0.0000e+00
Epoch 898/1000
4502/4502 [==============================] - 0s 57us/step - loss: 1.2192 - mean_absolute_error: 1.2192 - acc: 0.0000e+00 - val_loss: 1.5203 -

Epoch 933/1000
4502/4502 [==============================] - 0s 72us/step - loss: 1.2130 - mean_absolute_error: 1.2130 - acc: 0.0000e+00 - val_loss: 1.5372 - val_mean_absolute_error: 1.5372 - val_acc: 0.0000e+00
Epoch 934/1000
4502/4502 [==============================] - 0s 65us/step - loss: 1.2072 - mean_absolute_error: 1.2072 - acc: 0.0000e+00 - val_loss: 1.5337 - val_mean_absolute_error: 1.5337 - val_acc: 0.0000e+00
Epoch 935/1000
4502/4502 [==============================] - 0s 61us/step - loss: 1.2111 - mean_absolute_error: 1.2111 - acc: 0.0000e+00 - val_loss: 1.5476 - val_mean_absolute_error: 1.5476 - val_acc: 0.0000e+00
Epoch 936/1000
4502/4502 [==============================] - 0s 67us/step - loss: 1.2095 - mean_absolute_error: 1.2095 - acc: 0.0000e+00 - val_loss: 1.5203 - val_mean_absolute_error: 1.5203 - val_acc: 0.0000e+00
Epoch 937/1000
4502/4502 [==============================] - 0s 66us/step - loss: 1.2150 - mean_absolute_error: 1.2150 - acc: 0.0000e+00 - val_loss: 1.5175 -

Epoch 972/1000
4502/4502 [==============================] - 0s 64us/step - loss: 1.2025 - mean_absolute_error: 1.2025 - acc: 0.0000e+00 - val_loss: 1.5015 - val_mean_absolute_error: 1.5015 - val_acc: 0.0000e+00
Epoch 973/1000
4502/4502 [==============================] - 0s 62us/step - loss: 1.2016 - mean_absolute_error: 1.2016 - acc: 0.0000e+00 - val_loss: 1.5167 - val_mean_absolute_error: 1.5167 - val_acc: 0.0000e+00
Epoch 974/1000
4502/4502 [==============================] - 0s 71us/step - loss: 1.1995 - mean_absolute_error: 1.1995 - acc: 0.0000e+00 - val_loss: 1.5249 - val_mean_absolute_error: 1.5249 - val_acc: 0.0000e+00
Epoch 975/1000
4502/4502 [==============================] - 0s 63us/step - loss: 1.1976 - mean_absolute_error: 1.1976 - acc: 0.0000e+00 - val_loss: 1.5173 - val_mean_absolute_error: 1.5173 - val_acc: 0.0000e+00
Epoch 976/1000
4502/4502 [==============================] - 0s 65us/step - loss: 1.2001 - mean_absolute_error: 1.2001 - acc: 0.0000e+00 - val_loss: 1.5075 -

In [124]:
model.evaluate(validation_x, validation_y)

563/563 [==============================] - 0s 29us/step


[4.096431348505918, 4.096431348505918, 0.4191829491254491]

In [125]:
model.evaluate(test_x, test_y)

563/563 [==============================] - 0s 36us/step


[3.619178659750767, 3.619178659750767, 0.44049733527811974]

In [117]:
'''
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the model
model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10, callbacks=callbacks_list, verbose=0)
'''

[8.041566574340603, 8.041566574340603, 0.07104795715948617]